# Marine Carbonate System Visualizer

## Interactive Bjerrum Plot for Marine Chemistry

This tool visualizes the marine carbonate system using **Bjerrum plots**. 

**Features:**
- Interactive sliders for Total Alkalinity and DIC
- 4-panel visualization showing carbonate speciation
- Real-time calculations using PyCO2SYS
- Educational tool for marine chemistry courses

**Instructions:**
1. Adjust the sliders below to change Total Alkalinity and DIC values
2. Observe how the carbonate system responds in real-time
3. Use the Reset button to return to default values

## What does this tool show?

**Input parameters:**
- **Total alkalinity** (TA) in μmol/kg
- **Dissolved Inorganic Carbon** (DIC) in μmol/kg

**Fixed conditions:**
- **Salinity** = 35 PSU
- **Temperature** = 25°C

**Visualizations:**
1. **Bjerrum Plot** - Species fractions vs pH
2. **Current Composition** - Pie chart of species
3. **Saturation State** - Aragonite saturation
4. **System Analysis** - Detailed results

In [7]:
import PyCO2SYS as pyco2
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive_output, Button, Layout, HTML
from IPython.display import display

In [2]:
# 🔧 PyCO2SYS Configuration
# Modify these parameters to change calculation methods

CONFIG = {
    'salinity': 35,         # Salinity in PSU
    'temperature': 25,      # Temperature in °C
    'pressure': 0,          # Pressure in dbar (0 for surface)
    'opt_pH_scale': 1,      # pH scale: 1=Total, 2=Seawater, 3=Free, 4=NBS
    'opt_k_carbonic': 10    # Carbonic constants: 10=Waters et al.(2014) Real seawater
}

def set_config(**kwargs):
    """Update PyCO2SYS configuration easily
    
    Examples:
    set_config(opt_k_carbonic=8)              # Lueker et al. (2000)
    set_config(temperature=15, salinity=30)   # Different conditions
    set_config(opt_pH_scale=3)                # Free scale
    """
    for key, value in kwargs.items():
        if key in CONFIG:
            CONFIG[key] = value
            print(f"✓ Updated {key} = {value}")
        else:
            print(f"⚠ Unknown parameter: {key}")
            print(f"Available: {list(CONFIG.keys())}")
    
    print(f"\n📋 Current configuration: {CONFIG}")

# Common constants options for reference:
print("🔬 PyCO2SYS Configuration Ready!")
print(f"📋 Current settings: {CONFIG}")
print("\n📚 Common opt_k_carbonic options:")
print("   8  = Lueker et al. (2000) - Most common")
print("   10 = Waters et al. (2014) - Real seawater (default)")
print("   14 = Schockman & Byrne (2021) - Latest")
print("\n💡 Use: set_config(opt_k_carbonic=8) to change")

🔬 PyCO2SYS Configuration Ready!
📋 Current settings: {'salinity': 35, 'temperature': 25, 'pressure': 0, 'opt_pH_scale': 1, 'opt_k_carbonic': 10}

📚 Common opt_k_carbonic options:
   8  = Lueker et al. (2000) - Most common
   10 = Waters et al. (2014) - Real seawater (default)
   14 = Schockman & Byrne (2021) - Latest

💡 Use: set_config(opt_k_carbonic=8) to change


In [3]:
# ⚗️ Carbonate System Calculation Functions

def get_pyco2_params(alkalinity, DIC, pH=None):
    """Get standardized PyCO2SYS parameters using current CONFIG"""
    base_params = {
        'salinity': CONFIG['salinity'],
        'temperature': CONFIG['temperature'], 
        'pressure': CONFIG['pressure'],
        'opt_pH_scale': CONFIG['opt_pH_scale'],
        'opt_k_carbonic': CONFIG['opt_k_carbonic']
    }
    
    if pH is not None:
        # For pH-DIC calculations (Bjerrum plot)
        base_params.update({'par1': pH, 'par2': DIC, 'par1_type': 3, 'par2_type': 2})
    else:
        # For TA-DIC calculations (main)
        base_params.update({'par1': alkalinity, 'par2': DIC, 'par1_type': 1, 'par2_type': 2})
    
    return base_params

def compute_carbonate_system(alkalinity, DIC):
    """Compute carbonate system using PyCO2SYS with current CONFIG"""
    results = pyco2.sys(**get_pyco2_params(alkalinity, DIC))
    
    return {
        "pH_total": results["pH_total"], "pCO2": results["pCO2"],
        "bicarbonate": results["bicarbonate"], "carbonate": results["carbonate"],
        "omega_aragonite": results["saturation_aragonite"], **CONFIG
    }

def get_constants_from_current_system(data):
    """Extract K1, K2 constants from current equilibrium state"""
    pH, pCO2, HCO3, CO3 = data['pH_total'], data['pCO2'], data['bicarbonate'], data['carbonate']
    
    H = 10**(-pH)
    CO2 = pCO2 * 0.034
    
    # Calculate from equilibrium: K1 = [H][HCO3]/[CO2], K2 = [H][CO3]/[HCO3]
    K1 = (H * HCO3 / CO2) if CO2 > 0 else 10**(-6.0)
    K2 = (H * CO3 / HCO3) if HCO3 > 0 else 10**(-9.0)
    
    return K1, K2

print("⚗️ Calculation functions loaded!")

⚗️ Calculation functions loaded!


---

## 🚀 How to Use This Tool

### Step 1: Configure PyCO2SYS (Above)
- **Default settings** work for typical seawater conditions
- **To change**: Use `set_config(parameter=value)` in the configuration cell
- **Common changes**: 
  - `set_config(opt_k_carbonic=8)` - Use Lueker et al. (2000) constants
  - `set_config(temperature=15, salinity=30)` - Different conditions

### Step 2: Run the Interactive Interface (Below)
- Move the **sliders** to explore different TA and DIC values
- **Plots update automatically** showing:
  1. **Bjerrum Plot** - Species fractions vs pH
  2. **Composition** - Current system breakdown  
  3. **Saturation** - Aragonite saturation state
  4. **Analysis** - Detailed results with your chosen constants

### Step 3: Analyze Results
- **Red line** in Bjerrum plot shows current pH
- **Constants K1, K2** automatically match your PyCO2SYS settings
- **All calculations** use the same consistent parameters

---

In [4]:
# 📊 Plotting Functions - Bjerrum Diagram and Analysis

def create_bjerrum_plot(data):
    """Create 4-panel Bjerrum plot with system analysis"""
    plt.close('all')
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Marine Carbonate System Analysis', fontsize=16, fontweight='bold')
    
    colors = ['#440154', '#31688e', '#35b779', '#fde725']
    pH_range = np.linspace(4, 11, 40)
    
    # Get constants from current system state
    K1, K2 = get_constants_from_current_system(data)
    
    # Calculate speciation using analytical expressions
    H_values = 10**(-pH_range)
    denom = H_values**2 + K1*H_values + K1*K2
    
    alpha0 = H_values**2 / denom      # CO2
    alpha1 = K1*H_values / denom      # HCO3-
    alpha2 = K1*K2 / denom            # CO3-2
    
    # Plot 1: Bjerrum diagram
    ax1.plot(pH_range, alpha0, color=colors[0], linewidth=3, label='CO2')
    ax1.plot(pH_range, alpha1, color=colors[1], linewidth=3, label='HCO3-')
    ax1.plot(pH_range, alpha2, color=colors[2], linewidth=3, label='CO3-2')
    
    current_pH = data['pH_total']
    ax1.axvline(x=current_pH, color='red', linestyle='--', linewidth=2)
    ax1.text(current_pH + 0.1, 0.8, f'pH: {current_pH:.2f}', fontsize=10, color='red')
    ax1.set_xlabel('pH'), ax1.set_ylabel('Fraction of DIC')
    ax1.set_title('Bjerrum Plot'), ax1.legend(), ax1.grid(True, alpha=0.3)
    ax1.set_xlim(4, 11), ax1.set_ylim(0, 1)
    
    # Plot 2: Composition pie chart
    CO2_conc = data["pCO2"] * 0.034
    species = [CO2_conc, data["bicarbonate"], data["carbonate"]]
    ax2.pie(species, labels=['CO2', 'HCO3-', 'CO3-2'], autopct='%1.1f%%', colors=colors[:3])
    ax2.set_title(f'Composition at pH {current_pH:.2f}')
    
    # Plot 3: Saturation state
    omega = data["omega_aragonite"]
    ax3.bar(['Aragonite'], [omega], color=colors[3] if omega >= 1 else colors[0])
    ax3.axhline(y=1, color='black', linestyle='--')
    ax3.set_ylabel('Omega'), ax3.set_title('Saturation State')
    ax3.text(0, omega*0.7, f'Ω = {omega:.2f}', ha='center', fontsize=12)
    
    # Plot 4: System info
    ax4.axis('off')
    TA = data['bicarbonate'] + 2*data['carbonate']
    DIC = sum(species)
    
    info_text = f"""Input: TA={TA:.0f}, DIC={DIC:.0f} μmol/kg
    
Results: pH={current_pH:.2f}, pCO2={data['pCO2']:.0f} μatm
[HCO3-]={data['bicarbonate']:.0f}, [CO3-2]={data['carbonate']:.0f} μmol/kg
Ω_arag={omega:.2f}

Constants: K1={K1:.2e}, K2={K2:.2e}
Conditions: S={data['salinity']}, T={data['temperature']}°C
PyCO2SYS: pH_scale={data['opt_pH_scale']}, K_carbonic={data['opt_k_carbonic']}"""
    
    ax4.text(0.05, 0.95, info_text, transform=ax4.transAxes, fontsize=9,
             verticalalignment='top', fontfamily='monospace')
    
    plt.tight_layout()
    plt.show()
    return fig

print("📊 Plotting functions ready!")

📊 Plotting functions ready!


In [ ]:
# 🎛️ Interactive Interface - Clean and Simple

# Create sliders
sliders = {
    'alk': FloatSlider(value=2300, min=1800, max=3500, step=25,
                      description="Alkalinity (μmol/kg):", 
                      style={'description_width': 'initial'}, layout=Layout(width='600px')),
    'dic': FloatSlider(value=2020, min=1600, max=3500, step=25,
                      description="DIC (μmol/kg):",
                      style={'description_width': 'initial'}, layout=Layout(width='600px'))
}

# Update function (no duplicates)
def update_plots(alkalinity, DIC):
    """Update plot when sliders change"""
    data = compute_carbonate_system(alkalinity, DIC)
    create_bjerrum_plot(data)

# Reset function
def reset_values(b):
    """Reset sliders to default values"""
    sliders['alk'].value, sliders['dic'].value = 2300, 2020

# Reset button
reset_button = Button(description="🔄 Reset Values", button_style='info')
reset_button.on_click(reset_values)

# Display complete interface
display(VBox([
    HTML("<h3>🌊 Marine Carbonate System Explorer</h3>"),
    HTML("<p><em>📊 Complete 4-panel visualization using PyCO2SYS</em></p>"),

    sliders['alk'], 
    sliders['dic'], 
    reset_button,
    interactive_output(update_plots, {'alkalinity': sliders['alk'], 'DIC': sliders['dic']})
]))

print("🎯 Interactive carbonate system interface ready!")
print("📋 All calculations use PyCO2SYS with your chosen constants")
print("💡 Modify CONFIG above to change calculation parameters")

🎯 Interactive carbonate system interface ready!
📋 All calculations use PyCO2SYS with your chosen constants
💡 Modify CONFIG above to change calculation parameters


**Author:** Cardoso-Mohedano JG  
**Institution:** Instituto de Ciencias del Mar y Limnologia, UNAM, Estacion El Carmen  
**License:** [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)  
**ORCID:** [0000-0002-2918-972X](https://orcid.org/0000-0002-2918-972X)

---
This interactive tool allows exploration of the marine carbonate system using [PyCO2SYS](https://pyco2sys.readthedocs.io/en/latest/).  
Developed with support from [Claude AI](https://claude.ai) by Anthropic and [OpenAI ChatGPT](https://openai.com/chatgpt) and educational Python tools.